In [2]:
# ==================== SETTING UP THE MODEL FROM OPEN AI ====================

import os
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import OpenAI
from openai import OpenAI

# ==================== LODING THE ENV VARIABLES =========================
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# ==================== INISIATING THE CLIENT =============================
client = OpenAI()

In [4]:
messages = [
    {"role": "user",
     "content": """What's the weather like in San Francisco,
                   New York, and Las Vegass?"""
    }
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": """Get the current weather in a given
                              location.The default unit when not
                              specified is fahrenheit""",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": """The city and state,
                                        e.g. San Francisco, CA""",
                    },
                    "unit": {
                        "type": "string",
                        "default":"fahrenheit",
                        "enum": [ "fahrenheit", "celsius"],
                        "description": """The messuring unit for
                                          the temperature.
                                          If not explicitly specified
                                          the default unit is 
                                          fahrenheit"""
                    },
                },
                "required": ["location"],
            },
        },
    }
]

In [6]:
import json

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location. 
    The default unit when not specified is fahrenheit"""
    if "new york" in location.lower():
        return json.dumps(
            {"location": "New York", "temperature": "40", "unit": unit}
        )
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "50", "unit": unit}
        )
    elif "las vegas" in location.lower():
        return json.dumps(
            {"location": "Las Vegas", "temperature": "70", "unit": unit}
        )
    else:
        return json.dumps(
            {"location": location, "temperature": "unknown"}
        )

get_current_weather("New York")

'{"location": "New York", "temperature": "40", "unit": "fahrenheit"}'

In [7]:
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    tools=tools,
    tool_choice="auto", 
)

response_message = response.choices[0].message
print(f"this is the response_message = {response_message}")
tool_calls = response_message.tool_calls

if tool_calls:
    print (tool_calls)
    
    available_functions = {
        "get_current_weather": get_current_weather,
    } 
    messages.append(response_message)  
    
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  
    print (messages)

this is the response_message = ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xxWqXBgAoMvwfc3HB22alqmb', function=Function(arguments='{"location": "San Francisco, CA"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_HvrasxMup5FgSlYSjkdK5h4f', function=Function(arguments='{"location": "New York, NY"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_aIJK3pTPrSMjWzJ0DpUgzMAN', function=Function(arguments='{"location": "Las Vegas, NV"}', name='get_current_weather'), type='function')])
[ChatCompletionMessageToolCall(id='call_xxWqXBgAoMvwfc3HB22alqmb', function=Function(arguments='{"location": "San Francisco, CA"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_HvrasxMup5FgSlYSjkdK5h4f', function=Function(arguments='{"location": "New York, NY"}', name='get_current_weather'), type='function'), ChatC

In [9]:
second_response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages,
        )
print (second_response)

ChatCompletion(id='chatcmpl-9peAn4jBWy70IYU6SHkOfvuxtI5oQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Here's the current weather in the three cities:\n\n- **San Francisco, CA**: 50°F\n- **New York, NY**: 40°F\n- **Las Vegas, NV**: 70°F\n\nLet me know if you need more information!", role='assistant', function_call=None, tool_calls=None))], created=1722096181, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_ba606877f9', usage=CompletionUsage(completion_tokens=54, prompt_tokens=159, total_tokens=213))
